In [ ]:
# Important Installations
!pip install faiss-cpu gradio torchvision pillow
!pip install git+https://github.com/openai/CLIP.git
!pip install datasets

In [ ]:
# Importation des Bibliothéques
import os
import numpy as np
import torch
import clip
from PIL import Image
import faiss
import gradio as gr
from google.colab import drive

# Connexion à Google Drive
drive.mount('/content/drive')

# Spécification du dossier d’images
IMAGE_DIR = "/content/drive/MyDrive/ImageSearch"
os.makedirs(IMAGE_DIR, exist_ok=True)

# Chargement des chemins des Images
def load_images():
    return [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR)
            if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp"))]

# Chargement du modèle CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Construire la base de données des caractéristiques d'Images
image_paths = load_images()

# Extraction des caractéristiques (embeddings) des images avec CLIP
def extract_image_features(image_path):
    try:
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            feature = model.encode_image(image).cpu().numpy().flatten()
        return feature / np.linalg.norm(feature)
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Création de la base de données FAISS
image_features = np.array(
    [feature for img_path in image_paths if (feature := extract_image_features(img_path)) is not None],
    dtype=np.float32
)

if len(image_features) == 0:
    raise ValueError("No valid image features extracted. Check your image directory and formats.")

dimension = image_features.shape[1]
faiss_index = faiss.IndexFlatIP(dimension)  # Using inner product for cosine similarity
faiss.normalize_L2(image_features)
faiss_index.add(image_features)

# Encodage des requêtes textuelles
def extract_text_features(text):
    text_tokens = clip.tokenize([text]).to(device)
    with torch.no_grad():
        feature = model.encode_text(text_tokens).cpu().numpy().flatten()
    return feature / np.linalg.norm(feature)

# Recherche des images les plus pertinentes
def search_images(query_text, top_k=5, threshold=0.5):
    text_feature = extract_text_features(query_text)
    text_feature = np.expand_dims(text_feature, axis=0)
    distances, indices = faiss_index.search(text_feature, top_k)
    results = [(image_paths[i], 1 - distances[0][idx]) for idx, i in enumerate(indices[0])]

    # Filtrer les résultats pour n’afficher que ceux dont la similarité dépasse le seuil
    filtered_results = [(path, score) for path, score in results if score > threshold]

    return [(Image.open(path), f"Similarity: {score:.2f}") for path, score in filtered_results]

# Style CSS personnalisé pour l'interface
custom_css = """

/* Changer la couleur du fond de l'interface */
.gradio-container {
    background-color: #fefae0 !important;
    font-family: 'Segoe UI', sans-serif !important;
}

/* Changer la couleur du fond de l'interface en blanc */
.gallery-gallery {
    background-color: white !important;
    border-radius: 10px;
    padding: 10px;
    overflow: auto;
}

/* Assurer que chaque image a un fond blanc */
.gradio-gallery img {
    background-color: white !important;
    border-radius: 8px;
    padding: 5px;
    object-fit: cover;  /* Ensure images fill the available space */
}

/* Titre et texte */
h1, h2, h3 {
    color: #2c3e50;
    text-align: center;
}

/* Style des boutons */
button {
    background-color: #ffd6a5 !important;
    color: black !important;
    font-weight: bold;
    border: none !important;
    padding: 10px 20px;
    border-radius: 8px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}

button:hover {
    background-color: #ff9f1c !important;
}

input[type="text"] {
    background-color: #fff5d7 !important;
    border: 1px solid #e0c68c !important;
    padding: 10px;
    border-radius: 8px;
}

/* Pour le layout côte à côte */
.flex-row {
    display: flex;
    gap: 30px;
    align-items: flex-start;
    justify-content: center;
}

.flex-column {
    display: flex;
    flex-direction: column;
    gap: 15px;
    min-width: 300px;
}
"""

# Interface Gradio
with gr.Blocks(css=custom_css) as iface:
    gr.Markdown("# 🐾PetSeek")

    # Disposition en ligne (Search bar + Gallery)
    with gr.Row(elem_classes="flex-row"):
        with gr.Column(elem_classes="flex-column"):
            input_text = gr.Textbox(label="Search for an animal", placeholder="Ex: dog , cat...")
            search_btn = gr.Button("Search")
        with gr.Column(elem_classes="flex-column"):
            gallery = gr.Gallery(label="Results of the Search", elem_classes="gallery-container")

    # Action de recherche
    search_btn.click(fn=search_images, inputs=input_text, outputs=gallery)

iface.launch(debug=True)
